In [26]:
import numpy as np
import math

In [15]:
str = 'I AM A STUDENT.'

In [16]:
i=0
for c in str:
    if c.isalpha():
        code = ord(c)-ord('A')
        print(i, c, code)
        i+=1
    else:
        print(c)

0 I 8
 
1 A 0
2 M 12
 
3 A 0
 
4 S 18
5 T 19
6 U 20
7 D 3
8 E 4
9 N 13
10 T 19
.


In [17]:
apinkeys=[(2,3), (3,1), (4,2), (5,5)]
keyindex=0
i=0
encdata=""
for c in str:
    if c.isalpha():
        code = ord(c)-ord('A')
        print(i, c, code, 'key=', apinkeys[i%4])
        a = apinkeys[i%4][0]
        b = apinkeys[i%4][1]
        enc = (code*a+b)%26
        encchar = chr(enc+ord('A'))
        print('enc=', enc, ' str=', encchar)
        encdata+=encchar
        i+=1
    else: 
        print(c)
        encdata+=c
print(encdata)

0 I 8 key= (2, 3)
enc= 19  str= T
 
1 A 0 key= (3, 1)
enc= 1  str= B
2 M 12 key= (4, 2)
enc= 24  str= Y
 
3 A 0 key= (5, 5)
enc= 5  str= F
 
4 S 18 key= (2, 3)
enc= 13  str= N
5 T 19 key= (3, 1)
enc= 6  str= G
6 U 20 key= (4, 2)
enc= 4  str= E
7 D 3 key= (5, 5)
enc= 20  str= U
8 E 4 key= (2, 3)
enc= 11  str= L
9 N 13 key= (3, 1)
enc= 14  str= O
10 T 19 key= (4, 2)
enc= 0  str= A
.
T BY F NGEULOA.


In [27]:
blocksize = math.ceil(len(encdata)/4)*4
encdata2 = encdata+' '*(blocksize-len(encdata))

In [28]:
encdata2

'T BY F NGEULOA. '

In [32]:
rows = 4
cols = int(blocksize/4)

In [38]:
mat = np.asarray(list(encdata2))
print(mat)

['T' ' ' 'B' 'Y' ' ' 'F' ' ' 'N' 'G' 'E' 'U' 'L' 'O' 'A' '.' ' ']


In [41]:
mat = mat.reshape((rows, cols))

In [46]:
matlist = list(mat.T.reshape(-1))
cipher = ''.join(matlist)
print(cipher)

In [49]:
## Decrypt

In [50]:
blocksize = math.ceil(len(cipher)/4)*4
cipher2 = cipher+' '*(blocksize-len(cipher))

In [51]:
rows = 4
cols = int(blocksize/4)

In [52]:
mat = np.asarray(list(cipher2))
print(mat)

['T' ' ' 'G' 'O' ' ' 'F' 'E' 'A' 'B' ' ' 'U' '.' 'Y' 'N' 'L' ' ']


In [54]:
mat = mat.reshape((rows, cols))
mat

array([['T', ' ', 'G', 'O'],
       [' ', 'F', 'E', 'A'],
       ['B', ' ', 'U', '.'],
       ['Y', 'N', 'L', ' ']],
      dtype='<U1')

In [55]:
matlist = list(mat.T.reshape(-1))
cipher = ''.join(matlist)
print(cipher)

T BY F NGEULOA. 


In [56]:
i=0
for c in cipher:
    if c.isalpha():
        code = ord(c)-ord('A')
        print(i, c, code)
        i+=1
    else:
        print(c)

0 T 19
 
1 B 1
2 Y 24
 
3 F 5
 
4 N 13
5 G 6
6 E 4
7 U 20
8 L 11
9 O 14
10 A 0
.
 


In [64]:
i=0
encdata=""
for c in cipher:
    if c.isalpha():
        code = ord(c)-ord('A')
        print(i, c, code, 'key=', apinkeys[i%4])
        a = apinkeys[i%4][0]
        b = apinkeys[i%4][1]
        alpha = 0
        while (alpha+code-b)%a!=0:
            alpha += 26
        enc = int ( ((alpha+code-b)/a)%26 )
        print(enc)
        encchar = chr(enc+ord('A'))
        print('enc=', enc, ' str=', encchar)
        encdata+=encchar
        i+=1
    else: 
        print(c)
        encdata+=c
print(encdata)

0 T 19 key= (2, 3)
8
enc= 8  str= I
 
1 B 1 key= (3, 1)
0
enc= 0  str= A
2 Y 24 key= (4, 2)
12
enc= 12  str= M
 
3 F 5 key= (5, 5)
0
enc= 0  str= A
 
4 N 13 key= (2, 3)
5
enc= 5  str= F
5 G 6 key= (3, 1)
19
enc= 19  str= T
6 E 4 key= (4, 2)
7
enc= 7  str= H
7 U 20 key= (5, 5)
3
enc= 3  str= D
8 L 11 key= (2, 3)
4
enc= 4  str= E
9 O 14 key= (3, 1)
13
enc= 13  str= N
10 A 0 key= (4, 2)
6
enc= 6  str= G
.
 
I AM A FTHDENG. 


In [65]:
## *a+b operator test
str = "ABCDEFGHIJKLMNOPQRSTUVWXYZ"

In [66]:
# a,b = 2, 3

In [100]:
i=0
enccode = []
for c in str:
    if c.isalpha():
        code = ord(c)-ord('A')
        enc1 = code*16+2
        enc2 = enc1 % 26
        print(i, c, code, enc1, enc2)
        i+=1
        enccode.append(enc2)
enccode = np.asarray(enccode)
print(enccode)


0 A 0 2 2
1 B 1 18 18
2 C 2 34 8
3 D 3 50 24
4 E 4 66 14
5 F 5 82 4
6 G 6 98 20
7 H 7 114 10
8 I 8 130 0
9 J 9 146 16
10 K 10 162 6
11 L 11 178 22
12 M 12 194 12
13 N 13 210 2
14 O 14 226 18
15 P 15 242 8
16 Q 16 258 24
17 R 17 274 14
18 S 18 290 4
19 T 19 306 20
20 U 20 322 10
21 V 21 338 0
22 W 22 354 16
23 X 23 370 6
24 Y 24 386 22
25 Z 25 402 12
[ 2 18  8 24 14  4 20 10  0 16  6 22 12  2 18  8 24 14  4 20 10  0 16  6 22
 12]


In [101]:
enccode.sort()
print(enccode)

[ 0  0  2  2  4  4  6  6  8  8 10 10 12 12 14 14 16 16 18 18 20 20 22 22 24
 24]
